In [1]:
import sys
sys.path.append("../../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F
import argparse
import matplotlib

from tqdm import tqdm
import glob
from PIL import Image
import os
from datetime import datetime
import time
import math
from torch.autograd import Variable
from biotorch.module.biomodule import BioModule

from ANN import *
from visualization import *

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), 
                                            std=(3*0.2023, 3*0.1994, 3*0.2010))])

cifar_dset_train = torchvision.datasets.CIFAR100('../../../data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(cifar_dset_train, batch_size=20, shuffle=True, num_workers=0)

cifar_dset_test = torchvision.datasets.CIFAR100('../../../data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(cifar_dset_test, batch_size=20, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [49]:
device = "cuda"
criterion = torch.nn.MSELoss().to(device)
# criterion = torch.nn.CrossEntropyLoss()
activation = F.relu
architecture = [int(32*32*3), 1000, 500, 100]
model = BioModule(MLP(architecture, activation = activation, final_layer_activation = activation), 
                  mode = "fa", layer_config = {"type": "fa", "options": {"init": "kaiming"}}).to(device)
# model =MLP(architecture, activation = activation, final_layer_activation = False).to(device)
model

Module has been converted to fa mode:

The layer configuration was:  {'type': 'fa', 'options': {'init': 'kaiming'}}
- All the 3 <class 'torch.nn.modules.linear.Linear'> layers were converted successfully.


BioModule(
  (module): MLP(
    (linear_layers): ModuleList(
      (0): Linear(in_features=3072, out_features=1000, bias=True)
      (1): Linear(in_features=1000, out_features=500, bias=True)
      (2): Linear(in_features=500, out_features=100, bias=True)
    )
  )
)

In [50]:
for idx in range(len(model.module.linear_layers)):
    model.module.linear_layers[idx].scaling_factor *= 50
    model.module.linear_layers[idx].init_parameters()

In [5]:
evaluateClassification(model, train_loader, "cuda")

Accuracy :	 0.01068


0.01068

In [51]:
# # specify optimizer (stochastic gradient descent) and learning rate
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-5)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.001, nesterov=False)

lr_decay_step = 50
lr_decay = 0.9
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=lr_decay_step, gamma=lr_decay)
trn_acc_list = []
tst_acc_list = []

n_epochs = 20
for epoch_ in range(n_epochs):
    model.train()
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = Variable(x.to(device)), Variable(y.to(device))
        y_one_hot = F.one_hot(y, num_classes=100)
        optimizer.zero_grad()
        y_hat = model(x)
#         loss = criterion(y_hat,y) # Use this if criterion = torch.nn.CrossEntropyLoss()
        loss = criterion(y_hat,y_one_hot.to(torch.float32)) # Use this if criterion = torch.nn.MSELoss().to(device)
        # backward pass: compute gradient of the loss with respect to model parameters
        model.zero_grad()
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
    
    scheduler.step()
    trn_acc = evaluateClassification(model, train_loader, device, False)
    tst_acc = evaluateClassification(model, test_loader, device, False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

2500it [00:28, 87.24it/s]
9it [00:00, 85.54it/s]

Epoch : 1, Train Accuracy : 0.03152, Test Accuracy : 0.0301


2500it [00:29, 85.87it/s]
9it [00:00, 84.36it/s]

Epoch : 2, Train Accuracy : 0.03786, Test Accuracy : 0.0384


2500it [00:28, 87.77it/s]
8it [00:00, 78.48it/s]

...

Epoch : 18, Train Accuracy : 0.0609, Test Accuracy : 0.0547


2500it [00:30, 81.90it/s]
9it [00:00, 86.99it/s]

Epoch : 19, Train Accuracy : 0.06208, Test Accuracy : 0.0527


2500it [00:30, 81.80it/s]


Epoch : 20, Train Accuracy : 0.0618, Test Accuracy : 0.0538


In [ ]:
plot_convergence_plot(trn_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'MLP Train Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
plot_convergence_plot(tst_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'MLP Test Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)